In [1]:
import torch
from torchvision import datasets
from torchvision import transforms
import numpy as np

In [2]:
data_path = './data'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True)
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True)
tensor_cifar10 = datasets.CIFAR10(data_path, train=True, download=False, transform=transforms.ToTensor())

Files already downloaded and verified
Files already downloaded and verified


In [3]:
dir(transforms)

['CenterCrop',
 'ColorJitter',
 'Compose',
 'FiveCrop',
 'Grayscale',
 'Lambda',
 'LinearTransformation',
 'Normalize',
 'Pad',
 'RandomAffine',
 'RandomApply',
 'RandomChoice',
 'RandomCrop',
 'RandomErasing',
 'RandomGrayscale',
 'RandomHorizontalFlip',
 'RandomOrder',
 'RandomPerspective',
 'RandomResizedCrop',
 'RandomRotation',
 'RandomSizedCrop',
 'RandomVerticalFlip',
 'Resize',
 'Scale',
 'TenCrop',
 'ToPILImage',
 'ToTensor',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'functional',
 'transforms']

In [4]:
imgs = torch.stack([img_t for img_t, _ in tensor_cifar10], dim=3)
imgs.shape

torch.Size([3, 32, 32, 50000])

In [5]:
imgs.view(3, -1).mean(dim=1)

tensor([0.4915, 0.4823, 0.4468])

In [6]:
imgs.view(3, -1).std(dim=1)

tensor([0.2470, 0.2435, 0.2616])

In [7]:
transformed_cifar10 = datasets.CIFAR10(data_path, train=True, download=False, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4915, 0.4823, 0.4468), std=(0.2470, 0.2435, 0.2616))
]))

In [34]:
transformed_val_cifar10 = datasets.CIFAR10(data_path, train=False, download=False, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4915, 0.4823, 0.4468), std=(0.2470, 0.2435, 0.2616))
]))

In [35]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label]) for img, label in transformed_cifar10 if label in [0, 2]]
cifar2_val = [(img, label_map[label]) for img, label in transformed_val_cifar10 if label in [0, 2]]

In [19]:
import torch.nn as nn
import torch.optim as optim

In [20]:
n_out = 2
model = nn.Sequential(
    nn.Linear(3072, 512),
    nn.Tanh(),
    nn.Linear(512, n_out),
    nn.LogSoftmax(dim=1)
    )

In [22]:
learning_rate = 1e-2

In [23]:
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [24]:
loss_fn = nn.NLLLoss()

In [25]:
n_epochs = 100

for epoch in range(n_epochs):
    for img, label in cifar2:
        out = model(img.view(-1).unsqueeze(0))
        loss = loss_fn(out, torch.tensor([label]))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print("Epoch: %d, Loss %f" % (epoch, float(loss)))

In [28]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=True)

In [29]:
for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        loss = loss_fn(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 0.631998
Epoch: 1, Loss: 0.505895
Epoch: 2, Loss: 0.217186
Epoch: 3, Loss: 0.416558
Epoch: 4, Loss: 0.200614
Epoch: 5, Loss: 0.430290
Epoch: 6, Loss: 0.319100
Epoch: 7, Loss: 0.453715
Epoch: 8, Loss: 0.332388
Epoch: 9, Loss: 0.334141
Epoch: 10, Loss: 0.396666
Epoch: 11, Loss: 0.275633
Epoch: 12, Loss: 0.317735
Epoch: 13, Loss: 0.437850
Epoch: 14, Loss: 0.618088
Epoch: 15, Loss: 0.139701
Epoch: 16, Loss: 0.238316
Epoch: 17, Loss: 0.113065
Epoch: 18, Loss: 0.245610
Epoch: 19, Loss: 0.589082
Epoch: 20, Loss: 0.145723
Epoch: 21, Loss: 0.088561
Epoch: 22, Loss: 0.192270
Epoch: 23, Loss: 0.216492
Epoch: 24, Loss: 0.264257
Epoch: 25, Loss: 0.152803
Epoch: 26, Loss: 0.244084
Epoch: 27, Loss: 0.253490
Epoch: 28, Loss: 0.210529
Epoch: 29, Loss: 0.160064
Epoch: 30, Loss: 0.222224
Epoch: 31, Loss: 0.189593
Epoch: 32, Loss: 0.393194
Epoch: 33, Loss: 0.198774
Epoch: 34, Loss: 0.087401
Epoch: 35, Loss: 0.088975
Epoch: 36, Loss: 0.168372
Epoch: 37, Loss: 0.104950
Epoch: 38, Loss: 0.148

In [36]:
import torchvision.transforms as transforms

val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64, shuffle=False)

correct = 0
total = 0

to_tensor = transforms.ToTensor()

with torch.no_grad():
    for imgs, labels in val_loader:
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %f", correct / total)

Accuracy: %f 0.812


In [39]:
model = nn.Sequential(
        nn.Linear(3072, 1024),
        nn.Tanh(),
        nn.Linear(1024, 512),
        nn.Tanh(),
        nn.Linear(512, 128),
        nn.Tanh(),
        nn.Linear(128, 2)
        )

In [38]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=True)

In [40]:
loss_fn = nn.CrossEntropyLoss()

In [41]:
learning_rate = 1e-2

In [42]:
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [43]:
for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        loss = loss_fn(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 0.542710
Epoch: 1, Loss: 0.440551
Epoch: 2, Loss: 0.376583
Epoch: 3, Loss: 0.290062
Epoch: 4, Loss: 0.640769
Epoch: 5, Loss: 0.387951
Epoch: 6, Loss: 0.560414
Epoch: 7, Loss: 0.358832
Epoch: 8, Loss: 0.674810
Epoch: 9, Loss: 0.389153
Epoch: 10, Loss: 0.420622
Epoch: 11, Loss: 0.238805
Epoch: 12, Loss: 0.283217
Epoch: 13, Loss: 0.324534
Epoch: 14, Loss: 0.160171
Epoch: 15, Loss: 0.391469
Epoch: 16, Loss: 0.536314
Epoch: 17, Loss: 0.247964
Epoch: 18, Loss: 0.171954
Epoch: 19, Loss: 0.123365
Epoch: 20, Loss: 0.270261
Epoch: 21, Loss: 0.693502
Epoch: 22, Loss: 0.260981
Epoch: 23, Loss: 0.493363
Epoch: 24, Loss: 0.137405
Epoch: 25, Loss: 0.240633
Epoch: 26, Loss: 0.437690
Epoch: 27, Loss: 0.546059
Epoch: 28, Loss: 0.435931
Epoch: 29, Loss: 0.169506
Epoch: 30, Loss: 0.593583
Epoch: 31, Loss: 0.069936
Epoch: 32, Loss: 0.147766
Epoch: 33, Loss: 0.088593
Epoch: 34, Loss: 0.167835
Epoch: 35, Loss: 0.477304
Epoch: 36, Loss: 0.170764
Epoch: 37, Loss: 0.097795
Epoch: 38, Loss: 0.224

In [44]:
correct = 0
total = 0

to_tensor = transforms.ToTensor()

with torch.no_grad():
    for imgs, labels in val_loader:
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %f", correct / total)

Accuracy: %f 0.809
